<img src="img/python-logo-notext.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;"><b>Namespaces and Closures</b></div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>

# Namespaces

Variables and function names exist in a *namespace*.

- Global variables and function names are in the global namespace.
- Names imported with `import` exist in the imported namespace.
- Names defined within a function are in that function's namespace.
    - parameters
    - local variables

A function's namespace "disappears" at the end of the body.

In [ ]:
# Ohne Angabe der Namensräume, siehe nächste Folie
# fmt: off
a = 1

def f(x):
    # print(a) # Was passiert, wenn diese Zeile einkommentiert wird?
    a = x + 1
    print(a)

f(2)
print(a)
# print(x)
# fmt: on

In [ ]:
# fmt: off
a = 1         # Globaler Namespace

def f(x):     # Namespace von f - x ist im globalen Namespace *nicht* sichtbar
    a = x + 1 # Namespace von f - a ist im globalen Namespace *nicht* sichtbar
    print(a)  # Greift auf a aus dem Namespace von f zu

f(2)
print(a)      # Greift auf a aus dem globalen Namespace zu
# print(x)    # Fehler: x ist im Namespace von f
# fmt: on

In [ ]:
# fmt: off
a = 1

def f2(x):
    global a
    a = x + 1
    print(a)

f2(2)
print(a)
a = 5
print(a)
# fmt: on

In [ ]:
from dis import dis

In [ ]:
dis(f)

In [ ]:
dis(f2)

In [ ]:
# fmt: off
a = 1

def f_broken():
    # noinspection PyUnresolvedReferences
    print(a)
    a = 2
# fmt: on

In [ ]:
# f_broken()

In [ ]:
dis(f_broken)


*Note:* In Python bodies of `if` and `for` statements do not introduce a
nested namespaces; variables defined in these bodies belong to the surrounding
namespace.

In [ ]:
if True:
    x = 123

In [ ]:
print(x)

In [ ]:
for my_index in range(3):
    y = my_index

In [ ]:
print(my_index)
print(y)


# Functions

In Python functions are first class objects, i.e., they can be referenced by
variables, passed as arguments to functions, etc.

Functions have type `Callable`.

In [ ]:
print(open.__name__)

In [ ]:
from typing import Callable

In [ ]:
isinstance(list, Callable)

In [ ]:
def call_with_two_args(fun):
    print(f"Calling function {fun.__name__!r}")
    result = fun(2, 3)
    print(f"Result is {result}")
    return result

In [ ]:
call_with_two_args(pow)

In [ ]:
import operator

call_with_two_args(operator.add)

In [ ]:
call_with_two_args(operator.lt)

In [ ]:
def print_and_add(x, y):
    print(f"x = {x}, y = {y}")
    return x + y

In [ ]:
call_with_two_args(print_and_add)


## Mini-workshop "print truthy elements"

Write a function `print_truthy_elements(a_list: list, fun: Callable)` that
prints each element `x` of `a_list` for which `fun(x)` returns a true value.

Check the output for the list `example_values` and the functions
`greater_than_2()` and `less_than_10()`.

In [ ]:
example_values = [1, 2, 3, 9, 10]

In [ ]:
def greater_than_2(n):
    return n > 2

In [ ]:
def less_than_10(n):
    return n < 10

In [ ]:
from typing import Callable

In [ ]:
def print_truthy_elements(a_list: list, fun: Callable):
    for x in a_list:
        if fun(x):
            print(x)

In [ ]:
print_truthy_elements(example_values, greater_than_2)

In [ ]:
print_truthy_elements(example_values, less_than_10)

## Closures

In Python it is possible to define functions inside other functions. The inner
functions can access the variables of the outer function.

In [ ]:
from random import randint

In [ ]:
def generate_random_value():
    return randint(1, 4)

In [ ]:
generate_random_value()

In [ ]:
from dis import dis

In [ ]:
dis(generate_random_value)

In [ ]:
def make_and_call_nested_function():
    def return_random_value():
        return randint(1, 4)

    return return_random_value()

In [ ]:
make_and_call_nested_function()

In [ ]:
dis(make_and_call_nested_function)

In [ ]:
def make_and_return_nested_function():
    def return_random_value():
        return randint(1, 4)

    return return_random_value

In [ ]:
my_fun = make_and_return_nested_function()
my_fun

In [ ]:
my_fun()

In [ ]:
dis(make_and_return_nested_function)

In [ ]:
def make_closure_1():
    local_value = randint(1, 4)

    def return_local_value():
        return local_value

    return return_local_value

In [ ]:
my_closure_1 = make_closure_1()
my_closure_1

In [ ]:
my_closure_1()

In [ ]:
your_closure_1 = make_closure_1()
your_closure_1

In [ ]:
your_closure_1()

In [ ]:
dis(make_closure_1)

In [ ]:
def make_closure_2():
    local_value = randint(1, 10)

    def return_local_value():
        return local_value

    def inc_local_value():
        nonlocal local_value
        local_value += 1

    return return_local_value, inc_local_value

In [ ]:
get_value_1, inc_value_1 = make_closure_2()
get_value_2, inc_value_2 = make_closure_2()
get_value_1(), get_value_2()

In [ ]:
get_value_1(), get_value_2()

In [ ]:
inc_value_1()
get_value_1(), get_value_2()

In [ ]:
inc_value_2()
get_value_1(), get_value_2()

In [ ]:
dis(make_closure_2)


## Mini-workshop "mean computation"

Write a function `make_mean_fun()` that returns two closures

- a function `add_value(new_value: int)` that appends `new_value` to a list
  stored in a local variable `values` of `make_mean_fun()`
- a function `compute_mean()` that return the mean value of all values
  previously stored in `values`.

Do you have to use `nonlocal` to access `value`? Why, or why not?

Ensure that your implementation satisfies the provided test cases.

In [ ]:
def make_mean_fun():
    values: list[int] = []

    def add_value(new_value: int):
        values.append(new_value)

    def compute_mean():
        return sum(values) / len(values)

    return add_value, compute_mean


Test cases:

In [ ]:
add_value_1, compute_mean_1 = make_mean_fun()
add_value_2, compute_mean_2 = make_mean_fun()

In [ ]:
for i in range(10):
    add_value_1(i)

for i in range(2, 21, 4):
    add_value_2(i)

In [ ]:
assert compute_mean_1() == 4.5

In [ ]:
assert compute_mean_2() == 10.0


Write a function `make_mean_fun_2()` that returns closures with similar
functionality but stores only the number of added elements and their total
sum.

Do you have to use `nonlocal` to access the closure variables in this case?
Why, or why not?

In [ ]:
def make_mean_fun_2():
    sum_of_values: int = 0
    num_values: int = 0

    def add_value(new_value: int):
        nonlocal sum_of_values, num_values
        sum_of_values += new_value
        num_values += 1

    def compute_mean():
        return sum_of_values / num_values

    return add_value, compute_mean


Test cases:

In [ ]:
add_value_3, compute_mean_3 = make_mean_fun_2()
add_value_4, compute_mean_4 = make_mean_fun_2()

In [ ]:
for i in range(10):
    add_value_3(i)

In [ ]:
for i in range(2, 21, 4):
    add_value_4(i)

In [ ]:
assert compute_mean_3() == 4.5

In [ ]:
assert compute_mean_4() == 10.0


# Anonymous functions

For short functions that are only used in a single place it is often
inconvenient to provide a named function definition:

In [ ]:
example_values = [1, 2, 3, 9, 10]

In [ ]:
def greater_than_2(n):
    return n > 2

In [ ]:
def less_than_10(n):
    return n < 10

In [ ]:
from typing import Callable

In [ ]:
def print_truthy_elements(a_list: list, fun: Callable):
    for x in a_list:
        if fun(x):
            print(x)

In [ ]:
print_truthy_elements(example_values, greater_than_2)

In [ ]:
print_truthy_elements(example_values, less_than_10)


For these cases Python offers lambda expressions as a syntactically simpler
alternative:

In [ ]:
print_truthy_elements(example_values, lambda n: n > 2)

In [ ]:
print_truthy_elements(example_values, lambda n: n < 10)

In [ ]:
print_truthy_elements(example_values, lambda n: n % 2 == 0)

In [ ]:
call_with_two_args(lambda x, y: 2 * x + y**2)


Many functions for sequences work well with lambdas:

In [ ]:
filter(lambda n: n > 2, example_values)

In [ ]:
list(filter(lambda n: n > 2, example_values))


It is often more "Pythonic" to use comprehensions, however.

In [ ]:
[n for n in example_values if n > 2]